<a href="https://colab.research.google.com/github/PrswpkWirom/Depression-Detection-FinalProject/blob/main/DD_vanilla_MFFNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataloader, mini batch

In [28]:
import os
import pandas as pd
import numpy as np
import torch
import pickle
from google.colab import files
import torch
import torch.nn as nn
import torch.nn.functional as F


In [29]:
pickle_file_path = '/content/drive/MyDrive/DD_input_text_tensor_cpu.pkl'
input_text_tensor = torch.load(pickle_file_path)
print(input_text_tensor)


tensor([[[-0.0584,  0.0803, -0.0016,  ..., -0.0751, -0.0839, -0.0127],
         [-0.0734,  0.0904,  0.0002,  ..., -0.0699, -0.0782, -0.0195],
         [-0.0787,  0.0642, -0.0052,  ..., -0.0863, -0.0811, -0.0373],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0524,  0.0920, -0.0036,  ..., -0.0810, -0.0577, -0.0343],
         [-0.0517,  0.0909, -0.0059,  ..., -0.0802, -0.0536, -0.0325],
         [-0.0736,  0.0564,  0.0038,  ..., -0.1028, -0.0545, -0.0117],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0596,  0.1051,  0.0067,  ..., -0.0871, -0.0634, -0.0286],
         [-0.0597,  0.0772, -0.0034,  ..., -0

<ipython-input-29-1ce23ae293e7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  input_text_tensor = torch.load(pickle_file_path)


In [30]:
print(input_text_tensor.shape)

torch.Size([10, 169, 768])


# Fastformer Implementation

This fastformer module is obtained from github: https://github.com/wuch15/Fastformer

In [ ]:
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertSelfOutput, BertIntermediate, BertOutput
class AttentionPooling(nn.Module):
    def __init__(self, config):
        self.config = config
        super(AttentionPooling, self).__init__()
        self.att_fc1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.att_fc2 = nn.Linear(config.hidden_size, 1)
        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()


    def forward(self, x, attn_mask=None):
        bz = x.shape[0]
        e = self.att_fc1(x)
        e = nn.Tanh()(e)
        alpha = self.att_fc2(e)
        alpha = torch.exp(alpha)
        if attn_mask is not None:
            alpha = alpha * attn_mask.unsqueeze(2)
        alpha = alpha / (torch.sum(alpha, dim=1, keepdim=True) + 1e-8)
        x = torch.bmm(x.permute(0, 2, 1), alpha)
        x = torch.reshape(x, (bz, -1))
        return x

class FastSelfAttention(nn.Module):
    def __init__(self, config):
        super(FastSelfAttention, self).__init__()
        self.config = config
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" %
                (config.hidden_size, config.num_attention_heads))
        self.attention_head_size = int(config.hidden_size /config.num_attention_heads)
        self.num_attention_heads = config.num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        self.input_dim= config.hidden_size

        self.query = nn.Linear(self.input_dim, self.all_head_size)
        self.query_att = nn.Linear(self.all_head_size, self.num_attention_heads)
        self.key = nn.Linear(self.input_dim, self.all_head_size)
        self.key_att = nn.Linear(self.all_head_size, self.num_attention_heads)
        self.transform = nn.Linear(self.all_head_size, self.all_head_size)

        self.softmax = nn.Softmax(dim=-1)

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads,
                                       self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask):
        # batch_size, seq_len, num_head * head_dim, batch_size, seq_len
        batch_size, seq_len, _ = hidden_states.shape
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        # batch_size, num_head, seq_len
        query_for_score = self.query_att(mixed_query_layer).transpose(1, 2) / self.attention_head_size**0.5
        # add attention mask
        query_for_score += attention_mask

        # batch_size, num_head, 1, seq_len
        query_weight = self.softmax(query_for_score).unsqueeze(2)

        # batch_size, num_head, seq_len, head_dim
        query_layer = self.transpose_for_scores(mixed_query_layer)

        # batch_size, num_head, head_dim, 1
        pooled_query = torch.matmul(query_weight, query_layer).transpose(1, 2).view(-1,1,self.num_attention_heads*self.attention_head_size)
        pooled_query_repeat= pooled_query.repeat(1, seq_len,1)
        # batch_size, num_head, seq_len, head_dim

        # batch_size, num_head, seq_len
        mixed_query_key_layer=mixed_key_layer* pooled_query_repeat

        query_key_score=(self.key_att(mixed_query_key_layer)/ self.attention_head_size**0.5).transpose(1, 2)

        # add attention mask
        query_key_score +=attention_mask

        # batch_size, num_head, 1, seq_len
        query_key_weight = self.softmax(query_key_score).unsqueeze(2)

        key_layer = self.transpose_for_scores(mixed_query_key_layer)
        pooled_key = torch.matmul(query_key_weight, key_layer)

        #query = value
        weighted_value =(pooled_key * query_layer).transpose(1, 2)
        weighted_value = weighted_value.reshape(
            weighted_value.size()[:-2] + (self.num_attention_heads * self.attention_head_size,))
        weighted_value = self.transform(weighted_value) + mixed_query_layer

        return weighted_value


class FastAttention(nn.Module):
    def __init__(self, config):
        super(FastAttention, self).__init__()
        self.self = FastSelfAttention(config)
        self.output = BertSelfOutput(config)

    def forward(self, input_tensor, attention_mask):
        self_output = self.self(input_tensor, attention_mask)
        attention_output = self.output(self_output, input_tensor)
        return attention_output

class FastformerLayer(nn.Module):
    def __init__(self, config):
        super(FastformerLayer, self).__init__()
        self.attention = FastAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask):
        attention_output = self.attention(hidden_states, attention_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

class FastformerEncoder(nn.Module):
    def __init__(self, config, pooler_count=1):
        super(FastformerEncoder, self).__init__()
        self.config = config
        self.encoders = nn.ModuleList([FastformerLayer(config) for _ in range(config.num_hidden_layers)])
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # support multiple different poolers with shared bert encoder.
        self.poolers = nn.ModuleList()
        if config.pooler_type == 'weightpooler':
            for _ in range(pooler_count):
                self.poolers.append(AttentionPooling(config))
        logging.info(f"This model has {len(self.poolers)} poolers.")

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if isinstance(module, (nn.Embedding)) and module.padding_idx is not None:
                with torch.no_grad():
                    module.weight[module.padding_idx].fill_(0)
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self,
                input_embs,
                attention_mask,
                pooler_index=0):
        #input_embs: batch_size, seq_len, emb_dim
        #attention_mask: batch_size, seq_len, emb_dim

        extended_attention_mask = attention_mask.unsqueeze(1)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        batch_size, seq_length, emb_dim = input_embs.shape
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_embs.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
        position_embeddings = self.position_embeddings(position_ids)

        embeddings = input_embs + position_embeddings

        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        #print(embeddings.size())
        all_hidden_states = [embeddings]

        for i, layer_module in enumerate(self.encoders):
            layer_outputs = layer_module(all_hidden_states[-1], extended_attention_mask)
            all_hidden_states.append(layer_outputs)
        assert len(self.poolers) > pooler_index
        output = self.poolers[pooler_index](all_hidden_states[-1], attention_mask)

        return output

class Model(torch.nn.Module):

    def __init__(self,config):
        super(Model, self).__init__()
        self.config = config
        self.dense_linear = nn.Linear(config.hidden_size,4)
        self.word_embedding = nn.Embedding(len(word_dict),256,padding_idx=0)
        self.fastformer_model = FastformerEncoder(config)
        self.criterion = nn.CrossEntropyLoss()
        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if isinstance(module, (nn.Embedding)) and module.padding_idx is not None:
                with torch.no_grad():
                    module.weight[module.padding_idx].fill_(0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self,input_ids,targets):
        mask=input_ids.bool().float()
        embds=self.word_embedding(input_ids)
        text_vec = self.fastformer_model(embds,mask)
        score = self.dense_linear(text_vec)
        loss = self.criterion(score, targets)
        return loss, score

def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

# Attempt 1: Vanilla MSFastformer

In [13]:
# class AttentionPooling(nn.Module):
#     def __init__(self, config):
#         super(AttentionPooling, self).__init__()
#         self.config = config
#         self.att_fc1 = nn.Linear(config.hidden_size, config.hidden_size)
#         self.att_fc2 = nn.Linear(config.hidden_size, 1)
#         self.apply(self.init_weights)

#     def init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()

#     def forward(self, x, attn_mask=None):
#         bz = x.shape[0]
#         e = self.att_fc1(x)
#         e = nn.Tanh()(e)
#         alpha = self.att_fc2(e)
#         alpha = torch.exp(alpha)
#         if attn_mask is not None:
#             alpha = alpha * attn_mask.unsqueeze(2)
#         alpha = alpha / (torch.sum(alpha, dim=1, keepdim=True) + 1e-8)
#         x = torch.bmm(x.permute(0, 2, 1), alpha)
#         x = x.reshape(bz, -1)
#         return x

# class FastSelfAttention(nn.Module):
#     def __init__(self, config):
#         super(FastSelfAttention, self).__init__()
#         self.config = config
#         if config.hidden_size % config.num_attention_heads != 0:
#             raise ValueError(
#                 "The hidden size (%d) is not a multiple of the number of attention "
#                 "heads (%d)" %
#                 (config.hidden_size, config.num_attention_heads))
#         self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
#         self.num_attention_heads = config.num_attention_heads
#         self.all_head_size = self.num_attention_heads * self.attention_head_size

#         self.query = nn.Linear(config.hidden_size, self.all_head_size)
#         self.query_att = nn.Linear(self.all_head_size, self.num_attention_heads)
#         self.key = nn.Linear(config.hidden_size, self.all_head_size)
#         self.key_att = nn.Linear(self.all_head_size, self.num_attention_heads)
#         self.transform = nn.Linear(self.all_head_size, self.all_head_size)
#         self.softmax = nn.Softmax(dim=-1)

#         self.apply(self.init_weights)

#     def init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()

#     def transpose_for_scores(self, x):
#         new_x_shape = x.size()[:-1] + (self.num_attention_heads,
#                                        self.attention_head_size)
#         x = x.view(*new_x_shape)
#         return x.permute(0, 2, 1, 3)

#     def forward(self, q_input, k_input, attention_mask=None):
#         batch_size, seq_len, _ = q_input.shape
#         # Project queries and keys
#         mixed_query_layer = self.query(q_input)
#         mixed_key_layer = self.key(k_input)

#         # Compute query scores
#         query_for_score = self.query_att(mixed_query_layer).transpose(1, 2) / self.attention_head_size**0.5
#         if attention_mask is not None:
#             query_for_score = query_for_score + attention_mask

#         query_weight = self.softmax(query_for_score).unsqueeze(2)
#         query_layer = self.transpose_for_scores(mixed_query_layer)

#         pooled_query = torch.matmul(query_weight, query_layer).transpose(1, 2).reshape(
#             batch_size, 1, self.all_head_size)
#         pooled_query_repeat = pooled_query.repeat(1, seq_len, 1)

#         # Modulate keys with pooled queries
#         mixed_query_key_layer = mixed_key_layer * pooled_query_repeat

#         # Compute key scores
#         query_key_score = (self.key_att(mixed_query_key_layer) / self.attention_head_size**0.5).transpose(1, 2)
#         if attention_mask is not None:
#             query_key_score = query_key_score + attention_mask

#         query_key_weight = self.softmax(query_key_score).unsqueeze(2)
#         key_layer = self.transpose_for_scores(mixed_query_key_layer)
#         pooled_key = torch.matmul(query_key_weight, key_layer)

#         # Compute final attention output
#         weighted_value = (pooled_key * query_layer).transpose(1, 2)
#         weighted_value = weighted_value.reshape(
#             weighted_value.size()[:-2] + (self.num_attention_heads * self.attention_head_size,))
#         attention_output = self.transform(weighted_value) + mixed_query_layer

#         return attention_output

# class FastAttention(nn.Module):
#     def __init__(self, config):
#         super(FastAttention, self).__init__()
#         self.self = FastSelfAttention(config)
#         self.output = nn.Identity()  # Replace BertSelfOutput with Identity for simplicity

#     def forward(self, input_tensor, attention_mask):
#         self_output = self.self(input_tensor, attention_mask)
#         attention_output = self.output(self_output)  # Assuming residual connection is handled elsewhere
#         return attention_output

# class FastformerLayer(nn.Module):
#     def __init__(self, config):
#         super(FastformerLayer, self).__init__()
#         self.attention = FastAttention(config)
#         self.intermediate = nn.Linear(config.hidden_size, config.intermediate_size)
#         self.output = nn.Linear(config.intermediate_size, config.hidden_size)
#         self.LayerNorm1 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#         self.LayerNorm2 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.activation = nn.GELU()

#     def forward(self, hidden_states, attention_mask):
#         attention_output = self.attention(hidden_states, attention_mask)
#         attention_output = self.dropout(attention_output)
#         attention_output = self.LayerNorm1(hidden_states + attention_output)

#         intermediate_output = self.intermediate(attention_output)
#         intermediate_output = self.activation(intermediate_output)

#         layer_output = self.output(intermediate_output)
#         layer_output = self.dropout(layer_output)
#         layer_output = self.LayerNorm2(attention_output + layer_output)

#         return layer_output

# class FastformerEncoder(nn.Module):
#     def __init__(self, config):
#         super(FastformerEncoder, self).__init__()
#         self.config = config
#         self.encoders = nn.ModuleList([FastformerLayer(config) for _ in range(config.num_hidden_layers)])
#         self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
#         self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)

#         self.apply(self.init_weights)

#     def init_weights(self, module):
#         if isinstance(module, (nn.Linear, nn.Embedding)):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if isinstance(module, nn.Embedding) and module.padding_idx is not None:
#                 with torch.no_grad():
#                     module.weight[module.padding_idx].fill_(0)
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
#         if isinstance(module, nn.Linear) and module.bias is not None:
#             module.bias.data.zero_()

#     def forward(self, input_embs, attention_mask):
#         batch_size, seq_length, emb_dim = input_embs.shape
#         position_ids = torch.arange(seq_length, dtype=torch.long, device=input_embs.device)
#         position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
#         position_embeddings = self.position_embeddings(position_ids)

#         embeddings = input_embs + position_embeddings
#         embeddings = self.LayerNorm(embeddings)
#         embeddings = self.dropout(embeddings)

#         extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
#         extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
#         extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

#         hidden_states = embeddings
#         for layer_module in self.encoders:
#             hidden_states = layer_module(hidden_states, extended_attention_mask)

#         return hidden_states




In [14]:
# class CustomModel(nn.Module):
#     def __init__(self, config):
#         super(CustomModel, self).__init__()
#         self.config = config
#         self.layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#         self.fastformer = FastSelfAttention(config)
#         self.fc1 = nn.Linear(config.hidden_size, config.intermediate_size)
#         self.fc2 = nn.Linear(config.intermediate_size, config.hidden_size)
#         self.output_fc = nn.Linear(config.hidden_size, config.num_labels)
#         self.gelu = nn.GELU()
#         self.apply(self.init_weights)

#     def init_weights(self, module):
#         if isinstance(module, (nn.Linear)):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)

#     def compute_covariance(self, x):
#         # x: [batch_size, seq_len, hidden_size]
#         batch_size, seq_len, hidden_size = x.size()
#         x_centered = x - x.mean(dim=1, keepdim=True)
#         covariance = torch.bmm(x_centered.transpose(1, 2), x_centered) / (seq_len - 1 + 1e-5)
#         # Reshape to [batch_size, 1, hidden_size]
#         covariance = covariance.mean(dim=2, keepdim=True).permute(0, 2, 1)
#         return covariance  # [batch_size, 1, hidden_size]

#     def forward(self, x, attention_mask):
#         # x: [batch_size, seq_len, embedding_dim]
#         batch_size, seq_len, _ = x.size()

#         # Normalize x
#         x_norm = self.layernorm(x)

#         # Compute U_i for i = 1, 3, 5
#         U_indices = [1, 3, 5]
#         U_list = []
#         for i in U_indices:
#             if i <= seq_len:
#                 U_i = self.compute_covariance(x_norm[:, :i, :])
#             else:
#                 U_i = self.compute_covariance(x_norm)
#             U_list.append(U_i)

#         U1, U3, U5 = U_list

#         # Prepare attention mask
#         extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2).to(dtype=x.dtype)
#         extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

#         # Compute P
#         P1 = self.fastformer(U5, U3, extended_attention_mask)
#         P2 = self.fastformer(U3, U1, extended_attention_mask)
#         P3 = self.fastformer(U1, U5, extended_attention_mask)
#         P = P1 + P2 + P3
#         P_norm = self.layernorm(P)

#         # Compute I = FC(GELU(FC(Norm(P))))
#         I = self.fc2(self.gelu(self.fc1(P_norm)))

#         # Compute J = I + X
#         seq_len_I = I.size(1)
#         J = I + x[:, :seq_len_I, :]

#         # Final output y = FC(Norm(J))
#         J_norm = self.layernorm(J)
#         y = self.output_fc(J_norm)
#         y = y.mean(dim=1)  # Aggregate over sequence length

#         return y

# # Example usage:
# # Assuming you have input embeddings x of shape [batch_size, seq_len, embedding_dim]

# batch_size = 10
# sequence_length = 169  # Adjust as needed
# embedding_dim = 768

# # Create random input embeddings
# x = torch.randn(batch_size, sequence_length, embedding_dim)

# # Create attention mask
# attention_mask = torch.ones(batch_size, sequence_length)

# # Instantiate the configuration
# class Config:
#     def __init__(self):
#         self.hidden_size = 768
#         self.num_attention_heads = 12
#         self.num_hidden_layers = 12
#         self.intermediate_size = 3072
#         self.hidden_dropout_prob = 0.1
#         self.layer_norm_eps = 1e-12
#         self.initializer_range = 0.02
#         self.num_labels = 4
#         self.max_position_embeddings = 512
#         self.pooler_type = 'weightpooler'

# config = Config()

In [16]:
# batch_size = 10
# sequence_length = 1  # Adjust as needed
# embedding_dim = config.hidden_size

# x = input_text_tensor

# # Create attention mask
# attention_mask = torch.ones(batch_size, sequence_length)

# # Instantiate the model
# model = CustomModel(config)

# # Forward pass
# output = model(x, attention_mask)

# print("Output shape:", output.shape)  # Should be [batch_size, num_labels]

RuntimeError: shape '[10, 1, 768]' is invalid for input of size 921600

# Attempt 2: Also Vanilla MSFastformer

In [41]:
class FastSelfAttention(nn.Module):
    def __init__(self, config):
        super(FastSelfAttention, self).__init__()
        self.config = config
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" %
                (config.hidden_size, config.num_attention_heads))
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.num_attention_heads = config.num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        self.input_dim = config.hidden_size

        self.query = nn.Linear(self.input_dim, self.all_head_size)
        self.query_att = nn.Linear(self.all_head_size, self.num_attention_heads)
        self.key = nn.Linear(self.input_dim, self.all_head_size)
        self.key_att = nn.Linear(self.all_head_size, self.num_attention_heads)
        self.transform = nn.Linear(self.all_head_size, self.all_head_size)

        self.softmax = nn.Softmax(dim=-1)

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads,
                                       self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, context_states, attention_mask=None):
        # hidden_states: [batch_size, seq_len_hidden, hidden_size]
        # context_states: [batch_size, seq_len_context, hidden_size]
        batch_size, seq_len_hidden, _ = hidden_states.size()
        seq_len_context = context_states.size(1)

        # Compute queries and keys
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(context_states)

        # Compute attention scores
        query_for_score = self.query_att(mixed_query_layer).transpose(1, 2) / self.attention_head_size ** 0.5
        key_for_score = self.key_att(mixed_key_layer).transpose(1, 2) / self.attention_head_size ** 0.5

        # Compute attention scores
        attention_scores = torch.matmul(query_for_score.transpose(1, 2), key_for_score)  # [batch_size, seq_len_hidden, seq_len_context]

        # If attention_mask is provided, add it
        if attention_mask is not None:
            # attention_mask should be broadcastable to [batch_size, seq_len_hidden, seq_len_context]
            attention_scores += attention_mask

        # Compute attention weights
        attention_weights = self.softmax(attention_scores)

        # Compute context vector
        context_layer = torch.matmul(attention_weights, mixed_key_layer)  # [batch_size, seq_len_hidden, hidden_size]

        # Combine with value projection
        output = self.transform(context_layer) + mixed_query_layer

        return output

class FastAttention(nn.Module):
    def __init__(self, config):
        super(FastAttention, self).__init__()
        self.self = FastSelfAttention(config)
        self.output = nn.Linear(config.hidden_size, config.hidden_size)

    def forward(self, input_tensor, context_tensor, attention_mask=None):
        self_output = self.self(input_tensor, context_tensor, attention_mask)
        attention_output = self.output(self_output) + input_tensor
        return attention_output

class FastformerLayer(nn.Module):
    def __init__(self, config):
        super(FastformerLayer, self).__init__()
        self.attention = FastAttention(config)
        self.intermediate = nn.Linear(config.hidden_size, config.intermediate_size)
        self.output = nn.Linear(config.intermediate_size, config.hidden_size)
        self.activation = nn.GELU()
        self.layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(self, hidden_states, context_states, attention_mask=None):
        attention_output = self.attention(hidden_states, context_states, attention_mask)
        attention_output = self.layernorm(attention_output)
        intermediate_output = self.activation(self.intermediate(attention_output))
        layer_output = self.output(intermediate_output) + attention_output
        layer_output = self.layernorm(layer_output)
        return layer_output

In [42]:
class MSFastformer(nn.Module):
    def __init__(self, config):
        super(MSFastformer, self).__init__()
        self.config = config
        self.layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.fastformer = FastformerLayer(config)
        self.fc1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.fc2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.output_fc = nn.Linear(config.hidden_size, config.num_labels)
        self.gelu = nn.GELU()
        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, (nn.Linear)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def compute_covariance(self, x):
        # x: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = x.size()
        x_centered = x - x.mean(dim=1, keepdim=True)
        covariance = torch.bmm(x_centered.transpose(1, 2), x_centered) / (seq_len - 1 + 1e-5)
        # Reshape to [batch_size, seq_len_cov, hidden_size]
        covariance = covariance.mean(dim=2, keepdim=True).permute(0, 2, 1)
        return covariance  # [batch_size, 1, hidden_size]

    def forward(self, x, attention_mask):
        # x: [batch_size, seq_len, embedding_dim]
        batch_size, seq_len, _ = x.size()

        # Normalize x
        x_norm = self.layernorm(x)

        # Compute U_i for i = 1, 3, 5
        U_indices = [1, 3, 5]
        U_list = []
        for i in U_indices:
            if i <= seq_len:
                U_i = self.compute_covariance(x_norm[:, :i, :])
            else:
                U_i = self.compute_covariance(x_norm)
            U_list.append(U_i)

        U1, U3, U5 = U_list

        # Compute P
        P1 = self.fastformer(U5, U3, attention_mask=None)
        P2 = self.fastformer(U3, U1, attention_mask=None)
        P3 = self.fastformer(U1, U5, attention_mask=None)
        P = P1 + P2 + P3
        P_norm = self.layernorm(P)

        # Compute I = FC(GELU(FC(Norm(P))))
        I = self.fc2(self.gelu(self.fc1(P_norm)))

        # Compute J = I + x
        seq_len_I = I.size(1)
        J = I + x[:, :seq_len_I, :]

        # Final output y = FC(Norm(J))
        J_norm = self.layernorm(J)
        y = self.output_fc(J_norm)
        y = y.mean(dim=1)  # Aggregate over sequence length

        return y

In [45]:
class Config:
    def __init__(self,
                 hidden_size=768,
                 num_attention_heads=12,
                 intermediate_size=3072,
                 num_labels=2,
                 num_hidden_layers=12,
                 hidden_dropout_prob=0.1,
                 attention_probs_dropout_prob=0.1,
                 max_position_embeddings=512,
                 layer_norm_eps=1e-12,
                 initializer_range=0.02,
                 hidden_act="gelu",
                 pooler_type='weightpooler',
                 num_attention_layers=12):
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.num_labels = num_labels
        self.num_hidden_layers = num_hidden_layers
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.layer_norm_eps = layer_norm_eps
        self.initializer_range = initializer_range
        self.hidden_act = hidden_act
        self.pooler_type = pooler_type
        self.num_attention_layers = num_attention_layers

In [46]:
config = Config(hidden_size=768, intermediate_size=3072, num_labels=2)
model = MSFastformer(config)

batch_size = 10
seq_length = 169
hidden_dim = 768
x = input_text_tensor
attention_mask = torch.ones(batch_size, seq_length)

output = model(x, attention_mask)

print(output.shape)  # Should be [batch_size, num_labels]


torch.Size([10, 2])
